In [5]:
import pandas as pd

# Load the CQ4 dataset
cq4_data = pd.read_csv('CQ4_dataset.csv')

# Function to transform the dataset
def transform_dataset(df):
    # Pivot the data to have tests as columns
    df_pivot = df.pivot_table(
        index='id_student',
        columns='test',
        values='weighted_score',
        aggfunc='sum'
    ).reset_index()
    
    # Rename columns to match the required format
    df_pivot.columns = ['id_student', 'test1', 'test2', 'test3', 'test4']
    
    # Fill NaN values with 0 (in case any test was missed)
    df_pivot = df_pivot.fillna(0)
    
    # Calculate the final score as the sum of weighted scores
    df_pivot['final'] = df_pivot[['test1', 'test2', 'test3', 'test4']].sum(axis=1)
    
    # Merge with the original dataframe to get the pass column
    df_final = pd.merge(df_pivot, df[['id_student', 'pass']].drop_duplicates(), on='id_student')
    
    return df_final

# Transform the CQ4 dataset
cq4_transformed = transform_dataset(cq4_data)

# Display the resulting DataFrame
print(cq4_transformed.head())

# Save the transformed dataset
cq4_transformed.to_csv('CQ4_transformed_dataset.csv', index=False)


   id_student  test1  test2  test3  test4  final  pass
0       26315    6.0   14.4  21.70   21.7  63.80     1
1       32930    5.7   11.8  19.60   18.2  55.30     1
2       39208    7.4   13.8  25.90   24.5  71.60     1
3       41547    8.0   12.6   8.75   14.0  43.35     1
4       57285    7.0   14.4  27.30   17.5  66.20     1


In [6]:
from sklearn.model_selection import train_test_split

# Load the transformed dataset
cq4_transformed = pd.read_csv('CQ4_transformed_dataset.csv')

# Separate pass and fail instances
pass_data = cq4_transformed[cq4_transformed['pass'] == 1]
fail_data = cq4_transformed[cq4_transformed['pass'] == 0]

# Ensure equal amounts of pass and fail instances
fail_data_oversampled = fail_data.sample(n=len(pass_data), replace=True, random_state=42)

# Combine the balanced data
balanced_data = pd.concat([pass_data, fail_data_oversampled])

# Split the data into training (300), testing (200), and validation (200) sets
train_data, temp_data = train_test_split(balanced_data, test_size=0.4, stratify=balanced_data['pass'], random_state=42)
test_data, val_data = train_test_split(temp_data, test_size=0.5, stratify=temp_data['pass'], random_state=42)

# Ensure the correct sizes
train_data = train_data.sample(n=600, random_state=42)
test_data = test_data.sample(n=200, random_state=42)
val_data = val_data.sample(n=200, random_state=42)

# Save the datasets to CSV files
train_data.to_csv('CQ4_train.csv', index=False)
test_data.to_csv('CQ4_test.csv', index=False)
val_data.to_csv('CQ4_val.csv', index=False)

# Display the sizes of the datasets
print(f'Training set size: {len(train_data)}')
print(f'Testing set size: {len(test_data)}')
print(f'Validation set size: {len(val_data)}')


Training set size: 600
Testing set size: 200
Validation set size: 200


In [7]:
train_data

,id_student,test1,test2,test3,test4,final,pass
556,650565,5.1,9.6,16.10,12.95,43.75,0
710,679716,6.0,9.6,21.00,16.80,53.40,1
22,126769,4.0,8.0,21.00,21.00,54.00,1
605,653762,5.5,12.4,20.30,22.75,60.95,0
802,687037,7.6,13.4,24.85,25.20,71.05,1
...,...,...,...,...,...,...,...
761,683783,6.7,13.0,23.45,26.60,69.75,1
877,692175,6.0,13.6,21.00,25.55,66.15,0
511,647883,7.8,13.4,21.70,23.80,66.70,1
803,687060,6.4,11.2,21.00,14.00,52.60,0
